In [1]:
from glob import glob
import json
from tqdm import tqdm

In [2]:
selected = [
    '/home/husein/ssd3/instructions/synthetic-codealpaca-v1-chatgpt4.jsonl',
    '/home/husein/ssd3/instructions/synthetic-glaive_coder_raw_text.jsonl',
    '/home/husein/ssd3/instructions/synthetic-oss_instruct-decontaminated.jsonl'
]

In [3]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama',
    'bagian',
    'fungsionalitas',
]

indons = []
def found_word(s, words):
    global indons
    for i in range(len(words)):
        if words[i] in s:
            indons.append(s)
            return True
    return False

In [4]:
def replace_code(s):
    s = s.replace(' Kode ', ' Kod ')
    s = s.replace(' kode ', ' kod ')
    s = s.replace('Kode ', 'Kod ')
    s = s.replace('kodenya', 'kodnya')
    s = s.replace('Kodenya', 'Kodnya')
    s = s.replace('opkode', 'opkod')
    s = s.replace(' kode', ' kod')
    s = s.replace('dikodekan', 'dikodkan')
    s = s.replace('kodks', 'kodeks')
    s = s.replace('kode ', 'kod ')
    s = s.replace(' Kode', ' Kod')
    return s

In [5]:
english_news = glob('/home/husein/ssd3/google-translate-english-news/*.requested')
len(english_news)

32

In [6]:
english_texts = glob('/home/husein/ssd3/google-translate-english-texts/*.requested')
len(english_texts)

39

In [7]:
malay_news = glob('/home/husein/ssd3/google-translate-malay-news/*.requested')
len(malay_news)

22

In [8]:
hansard = glob('/home/husein/ssd3/google-translate-malaysia-hansard/*.requested')
len(hansard)

5

In [9]:
ens = [english_news, english_texts]
mss = [malay_news, hansard]

In [10]:
def reject(k):
    if 'return JSON' in k:
        return
    if 'AI language model' in k:
        return
    if 'model bahasa AI' in k:
        return
    return True

In [11]:
with open('train.json', 'w') as fopen_jsonl:
    
    for f in selected:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                en = l['instruction_en']
                ms = l['instruction_ms']

                if not en or not ms:
                    continue

                if not reject(en) or not reject(ms):
                    continue
                
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue
                
                if len(en) and len(en.split()) < 1800 and len(ms) and len(ms.split()) < 1800:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                

In [12]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/gpt4all-1.3/translated-gpt4all-filtered-noncode.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            try:
                en = l['src']['prompt']
                ms = l['r'][0]['result']
                
                if not reject(en) or not reject(ms):
                    continue
                    
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue

                if len(en) and len(en.split()) < 1800 and len(ms) and len(ms.split()) < 1800:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

            except:
                pass
            
            try:
                en = l['src']['response']
                ms = l['r'][1]['result']
                
                if not reject(en) or not reject(ms):
                    continue
                    
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue
                
                if len(en) and len(en.split()) < 1800 and len(ms) and len(ms.split()) < 1800:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            except:
                pass

439509it [00:44, 9980.37it/s] 


In [13]:
!wc -l train.json

2188918 train.json


In [14]:
with open('train.json', 'a') as fopen_jsonl:
    for f in [
        '/home/husein/ssd3/translation/processed-ms-id.jsonl', 
        '/home/husein/ssd3/translation/processed-ms-jw.jsonl'
    ]:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                left = l['left']
                right = l['right']
                if len(left) and len(left.split()) < 1800 and len(right) and len(right.split()) < 1800:
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/translation/process-bjn-Latn.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            left = l['left']
            chatgpt = l['chatgpt']
            google = l['google']
            
            if not reject(chatgpt) or not reject(google):
                continue
            
            if len(left) and len(left.split()) < 1800 and len(google) and len(google.split()) < 1800:
                d = {"translation": {"src": left, "tgt": google, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if len(chatgpt) and len(chatgpt.split()) < 1800 and len(google) and len(google.split()) < 1800:
                d = {"translation": {"src": chatgpt, "tgt": google, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

2496675it [00:19, 128751.77it/s]
1683631it [00:09, 171204.35it/s]
958405it [00:10, 92013.37it/s] 


In [15]:
!wc -l train.json

8286032 train.json


In [16]:
with open('train.json', 'a') as fopen_jsonl:
    with open('gpt4all-code.jsonl') as fopen:
        for l in tqdm(fopen):
            try:
                l = json.loads(l)
                left = l['prompt'][0]
                right = l['prompt'][1]
                
                right = replace_code(right)
        
                if found_word(right, rejected_words):
                    continue
                
                if len(left) and len(right):
                    d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                
                left = l['response'][0]
                right = l['response'][1]
                
                right = replace_code(right)
        
                if found_word(right, rejected_words):
                    continue
                
                if len(left) and len(right):
                    d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
            except Exception as e:
                print(e)

75777it [00:10, 7498.74it/s] 


In [17]:
!wc -l train.json

8555444 train.json


In [18]:
import random

with open('train.json', 'a') as fopen_jsonl:
    for fs in ens:
        for f in fs:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    
                    if random.random() > 0.2:
                        continue
                        
                    data = json.loads(l)
                    en = data['src']
                    ms = data['r']['result']

                    if len(en) and len(en.split()) < 1800 and len(ms) and len(ms.split()) < 1800:
                        d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
                        d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
    for fs in mss:
        for f in fs:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    
                    if 'malaysia-hansard' not in f and random.random() > 0.4:
                        continue
                    
                    data = json.loads(l)
                    if isinstance(data['src'], dict):
                        s = data['src']['cleaned']
                    else:
                        s = data['src']
                    ms = s
                    en = data['r']['result']

                    if len(en) and len(en.split()) < 1800 and len(ms) and len(ms.split()) < 1800:
                        
                        d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
                        d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

99999it [00:00, 304843.91it/s]
100000it [00:00, 548406.22it/s]
100000it [00:00, 540987.61it/s]
18837it [00:00, 529477.51it/s]
100000it [00:00, 516588.89it/s]
100000it [00:00, 545894.50it/s]
100000it [00:00, 545702.74it/s]
100000it [00:00, 539987.46it/s]
100000it [00:00, 534729.94it/s]
100000it [00:00, 546295.51it/s]
100000it [00:00, 550466.83it/s]
100000it [00:00, 539278.60it/s]
100000it [00:00, 524173.34it/s]
100000it [00:00, 384163.43it/s]
100000it [00:00, 481753.94it/s]
100000it [00:00, 438353.23it/s]
100000it [00:00, 538786.06it/s]
100000it [00:00, 539827.61it/s]
100000it [00:00, 545477.77it/s]
100000it [00:00, 537957.50it/s]
100000it [00:00, 541690.48it/s]
100000it [00:00, 542370.64it/s]
100000it [00:00, 365940.83it/s]
100000it [00:00, 539326.44it/s]
100000it [00:00, 264020.03it/s]
100000it [00:00, 518760.61it/s]
100000it [00:00, 532502.52it/s]
100000it [00:00, 541452.03it/s]
100000it [00:00, 429112.03it/s]
100000it [00:00, 373072.75it/s]
100000it [00:00, 529669.44it/s]
100000it [

In [19]:
!wc -l train.json

13349226 train.json


In [20]:
def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return words[i]
    return False

In [21]:
!shuf train.json > shuffled-train.json

In [22]:
!head -n 1 shuffled-train.json

{"translation": {"src": "Jumlah karya wanita pada akhir periode atau rata-rata jumlah karya wanita selama periode yang bersangkutan.", "tgt": "Bilangan kerja wanita pada akhir tempoh atau purata bilangan kerja wanita dalam tempoh tersebut.", "prefix": "terjemah ke Melayu: "}}


In [23]:
# !pip3.8 install mosaicml-streaming

In [24]:
from streaming import MDSWriter, LocalDataset

columns = {
    'src': 'str',
    'tgt': 'str',
    'prefix': 'str',
}
hashes = 'sha1', 'xxh64'

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [25]:
!rm -rf mosaic

In [26]:
with MDSWriter(out='mosaic', columns=columns, compression=None, hashes=hashes) as out:
    with open('shuffled-train.json') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)['translation']
            out.write(l)

13349226it [01:50, 121236.19it/s]


In [27]:
dataset = LocalDataset('mosaic')

In [28]:
len(dataset)

13349226

In [29]:
len(indons)

15700

In [30]:
for i in indons:
    if 'pertama-tama' in i:
        print(i)
        break

Terjemahan ke dalam Bahasa Melayu

Melaksanakan kalkulator asas yang mengambil persamaan sebagai rentetan input, mengendalikan pelbagai operator (+, -, *, /) dan hasil output. Jika persamaan tidak sah atau menghasilkan kesalahan seperti pembahagian dengan sifar, kembalikan mesej kesalahan yang sesuai.

Perhatikan kod Python yang rosak berikut. Perbaiki ini sehingga ia memenuhi spesifikasi yang dinyatakan di atas:

```Python
def calculate(equation: str):
    return eval(equation)
```

Sebagai contoh, diberikan rentetan "3 + 3 * 2", skrip anda harus mengembalikan 9, bukan 12. Berikan kes ujian yang merangkumi setiap operator sekurang-kurangnya sekali.

---

Berikut adalah versi yang lebih baik daripada kod Python:

```Python
import re

def calculate(equation: str):
    # Periksa jika persamaan adalah sah
    if not re.match("[0-9+*/ -]*$", equation):
        return "ERROR: Invalid equation"
    
    # Bahagikan persamaan dan nilai hasilnya
    try:
        return eval(''.join(re.split('(

In [31]:
dataset[10]

{'prefix': 'terjemah ke Melayu: ',
 'src': 'Ryosuke Kojima Ryosuke Kojima memulai karir profesionalnya pada tahun 2019 saat masih bersama Oita Trinita.',
 'tgt': 'Ryosuke Kojima Ryosuke Kojima memulakan kerjaya profesionalnya pada tahun 2019 ketika bersama Oita Trinita.'}

In [32]:
rm -rf train.jsonl shuffled-train.jsonl

In [33]:
!du -hs mosaic

8.8G	mosaic


In [34]:
!cp mosaic/* ~/ssd3/mosaic-standard-translation